In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    FileSearchTool,
)
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
project_client.agents.list_files()['data']

[{'object': 'file', 'id': 'assistant-soO8m1jYtYMeLt9QIq8dqgV2', 'purpose': 'assistants', 'filename': 'azureml://subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourcegroups/mmai04-rg/workspaces/mmai-hub04-prj01-fvye/datastores/workspaceblobstore/paths/LocalUpload/d6b40c5c4bae2d7022f0fb660f2773d4/product_info_1.md', 'bytes': 10999, 'created_at': 1735604056, 'status': 'processed', 'status_details': None}]

In [4]:
for f in project_client.agents.list_files()['data']:
    print(f"deleting {f.filename}...")
    project_client.agents.delete_file(f.id)

deleting azureml://subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourcegroups/mmai04-rg/workspaces/mmai-hub04-prj01-fvye/datastores/workspaceblobstore/paths/LocalUpload/d6b40c5c4bae2d7022f0fb660f2773d4/product_info_1.md...


In [5]:
file = project_client.agents.upload_file_and_poll(file_path="./product_info_1.md", purpose="assistants")
print(f"Uploaded file: {file}")

Uploaded file: {'object': 'file', 'id': 'assistant-p5GGjAesPBP0WwtkQ3lh4Ycm', 'purpose': 'assistants', 'filename': 'product_info_1.md', 'bytes': 10999, 'created_at': 1735604119, 'status': 'processed', 'status_details': None}


In [6]:
vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store: {vector_store}")

Created vector store: {'id': 'vs_9yI4NaEVjIz9opOMNx7BO8is', 'object': 'vector_store', 'name': 'my_vectorstore', 'status': 'completed', 'usage_bytes': 17143, 'created_at': 1735604121, 'file_counts': {'in_progress': 0, 'completed': 1, 'failed': 0, 'cancelled': 0, 'total': 1}, 'metadata': {}, 'expires_after': None, 'expires_at': None, 'last_active_at': 1735604121}


In [7]:
# Create file search tool with resources followed by creating agent

file_search = FileSearchTool(vector_store_ids=[vector_store.id])

In [8]:
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="Hello, you are helpful assistant and can search information from uploaded files",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_lOTLNLSDMXU3ViIXZKwkmiLB


In [9]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

Created thread: {'id': 'thread_4dw2WutGfbaN6KRWV6ZFsCPD', 'object': 'thread', 'created_at': 1735604123, 'metadata': {}, 'tool_resources': {}}


In [10]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello, how much for the TrailMaster?"
)
print(f"Created message: {message}")

Created message: {'id': 'msg_ABcO8nFRLzl0Zfca3p6kLoHZ', 'object': 'thread.message', 'created_at': 1735604124, 'assistant_id': None, 'thread_id': 'thread_4dw2WutGfbaN6KRWV6ZFsCPD', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, how much for the TrailMaster?', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [11]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}.\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.
Run: {'id': 'run_pGYoNhEqA4JbJLfGtJ63U4HF', 'object': 'thread.run', 'created_at': 1735604124, 'assistant_id': 'asst_lOTLNLSDMXU3ViIXZKwkmiLB', 'thread_id': 'thread_4dw2WutGfbaN6KRWV6ZFsCPD', 'status': 'completed', 'started_at': 1735604124, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735604127, 'required_action': None, 'last_error': None, 'model': 'gpt-4', 'instructions': 'Hello, you are helpful assistant and can search information from uploaded files', 'tools': [{'type': 'file_search', 'file_search': {'ranking_options': {'ranker': 'default_2024_08_21', 'score_threshold': 0.0}}}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 5201, 'completion_tokens': 36, 'total_tokens': 5237}, 'response_format': 'auto', 't

In [12]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
messages

{'object': 'list', 'data': [{'id': 'msg_LyXynJWLeAS8iOgIrNlvQqsY', 'object': 'thread.message', 'created_at': 1735604126, 'assistant_id': 'asst_lOTLNLSDMXU3ViIXZKwkmiLB', 'thread_id': 'thread_4dw2WutGfbaN6KRWV6ZFsCPD', 'run_id': 'run_pGYoNhEqA4JbJLfGtJ63U4HF', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The price of the TrailMaster X4 Tent is $250【4:0†source】.', 'annotations': [{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 44, 'end_index': 56, 'file_citation': {'file_id': 'assistant-p5GGjAesPBP0WwtkQ3lh4Ycm'}}, {'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 44, 'end_index': 56, 'file_citation': {'file_id': 'assistant-p5GGjAesPBP0WwtkQ3lh4Ycm'}}]}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_ABcO8nFRLzl0Zfca3p6kLoHZ', 'object': 'thread.message', 'created_at': 1735604124, 'assistant_id': None, 'thread_id': 'thread_4dw2WutGfbaN6KRWV6ZFsCPD', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'He

In [13]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: The price of the TrailMaster X4 Tent is $250【4:0†source】.


In [14]:
# Print citations from the messages
for citation in messages.file_citation_annotations:
    print(citation)

{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 44, 'end_index': 56, 'file_citation': {'file_id': 'assistant-p5GGjAesPBP0WwtkQ3lh4Ycm'}}
{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 44, 'end_index': 56, 'file_citation': {'file_id': 'assistant-p5GGjAesPBP0WwtkQ3lh4Ycm'}}


# START teardown

In [15]:
# Delete the file when done
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_file(file_id=file.id)
print("Deleted file")

Deleted vector store
Deleted file


In [16]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_lOTLNLSDMXU3ViIXZKwkmiLB)...


# HIC SUNT LEONES